# 符合Python风格的对象

----
基于 Python 的数据模型，自定义类型可以实现和内置类型一样自然的行为，实际上靠的是 **鸭子类型**。
> 鸭子类型：按照预定行为实现对象所需的方法

## 0、最初的Vector类

In [1]:
from math import hypot 	 # 两个数平方和开方

class Vector():
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):     # 字符串表示形式
        return 'Vector(%r, %r)' % (self.x, self.y)

    def __abs__(self):
        return hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))
        # OR return bool(self.x or self.y)

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

----

## 1、对象表现形式

Python 提供了两种获取对象的**字符串表现形式**的标准方式：
> `repr()` ：便于开发者理解的方式返回对象字符串表现形式，由 `__repr__` 特殊方法提供支持     
> `str()` ：便于用户理解的方式返回对象字符串表现形式，由 `__str__` 特殊方法提供支持

为了给对象提供其他的表现形式，还涉及到另外两个特殊方法：
> `__bytes__` ：与 `__str__` 类似；`bytes()` 调用它获取对象的 **字节序列表现形式**   
> `__format__` ：被内置的 `format()` 函数和 `str.format()` 方法调用，使用特殊的格式代码显示对象的字符串表现形式

! PS:
> Python3 中，`__repr__`、`__str__`、`__format__` 都必须返回 **Unicode 字符串** (`str` 类型)；`__bytes__` 返回 **字节序列** (`bytes` 类型)

----

## 2、Vector v0

In [43]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    # 尽早转换为float，尽早捕获错误，防止传入非法参数
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    # 实现可迭代，用于拆包
    def __iter__(self):
        return (i for i in (self.x, self.y))    # OR yield self.x; yield self.y
    
    # !r 获取各个分量的表现形式
    def __repr__(self):
        class_name = type(self).__name__     # OR self.__class__.__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    # 因为可迭代，可以根据实例得到元组
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))   # 可迭代，可以根据实例得到数组
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(*self)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod   # 定义类方法：从字节序列转换成该类的实例
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)    # 传入 octets 字节序列创建一个内存视图，使用 typecode 转换
        return cls(*memv)                          # 拆包内存视图得到一对参数传入构造方法

In [44]:
v1 = Vector2d(3, 4)

In [45]:
# 无需调用读值方法，直接通过属性访问 Vertor 实例的分量
print(v1.x, v1.y)

3.0 4.0


In [46]:
# 因为可迭代，可以拆包成变量元组
x, y = v1
x, y

(3.0, 4.0)

In [47]:
# repr 字符串表现形式
v1

Vector2d(3.0, 4.0)

In [48]:
# 使用 eval 表明 repr 函数调用实例得到的是对构造方法的准确表述
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [49]:
# str 字符串表现形式
print(v1)

(3.0, 4.0)


In [50]:
# 二进制表现形式
bytes(v1)

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [51]:
# Vertor模值
abs(v1)

5.0

In [52]:
# 布尔值
bool(v1), bool(Vector2d(0, 0))

(True, False)

In [53]:
# 字节序列转换成实例
v2 = Vector2d.frombytes(bytes(v1))
v2

Vector2d(3.0, 4.0)

----

## 3、`classmethod` 和 `staticmethod`

两者都是装饰器
> `classmethod` ：定义操作类的方法（类方法），而不是操作实例；类方法的第一个参数是类本身；最常见的用途是定义备选构造方法（Vector v0 中的`frombytes` 方法）    
> `staticmethod` ：静态方法，其实就是普通函数，只不过在类的定义体中定义，而不在模块层定义

In [54]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args

In [59]:
# 类方法的第一个参数始终是类本身
Demo.klassmeth(), Demo.klassmeth('spam')

((__main__.Demo,), (__main__.Demo, 'spam'))

In [60]:
# 静态方法的行为和普通函数相似
Demo.statmeth(), Demo.statmeth('spam')

((), ('spam',))

----

## 4、格式化显示

- 内置的 `format()` 函数和 `str.format()` 方法把各个类型的格式化方式委托给相应的 `__format__(format_spec)` 方法。    
- `format_spec` —— 格式说明符，使用**格式规范微语言**（Format Specification Mini-Language）表示法：
> `format(obj, format_spec)` 的第二个参数      
> `str.format()` 方法的格式字符串中，`{}` 里冒号之后的部分    

它和 % 格式化方法 % 后面的部分基本一样，但是有所不同（如对齐方式）       
详细的格式化符号及用法见 [官方文档](https://docs.python.org/3/library/string.html#formatstrings "Format String Syntax") 和 [fat39 的博客](https://www.cnblogs.com/fat39/p/7159881.html "Python 格式化输出")

In [61]:
brl = 1 / 2.43
brl

0.4115226337448559

In [63]:
format(brl, '0.4f')   # 0可以省略

'0.4115'

In [68]:
'1 BRL = {rate:^15.2e} USD'.format(rate=brl)

'1 BRL =    4.12e-01     USD'

上例中的 `0.4f` 和 `^15.2e`（取小数点后两位，科学计数法表示，居中并占15个位置单位）就是 `format_spec`

> 类似于 `{0.attr:5.3e}` 这样的格式字符串包括两个部分：     
> - 冒号前的是 **字段名**，可以是数字表示，也可以是有具体含义的名称，如果它是某个类的实例并且要使用它的某个属性，可以使用 `.` 来获取
> - 冒号后的就是 **格式说明符** (`format_spec`)

#### 4 种十进制转十六进制的写法

In [69]:
# 1. hex 内置函数
hex(100)

'0x64'

In [74]:
# 2. % 格式化
'%x' % 100

'64'

In [71]:
# 3. format 函数
format(100, 'x')

'64'

In [72]:
# 4. str.format() 方法
'{:x}'.format(100)

'64'

二进制、八进制、十进制、十六进制 之间都可以用上述类似的方法相互转换。格式规范微语言为这些内置类型提供了专用的表示代码（如十六进制的 x）

> 格式规范微语言是可扩展的，每个类可以自行决定如何解释 `format_spec` 参数      
> 如 `datetime` 模块中的类，它们的 `__format__` 方法使用的格式代码和 `strftime()` 函数一样

In [75]:
from datetime import datetime
now = datetime.now()

In [80]:
format(now, '%Y-%m-%d  %H:%M:%S')

'2018-12-28  20:52:25'

In [83]:
"It's now {:%I:%M %p}".format(now) 

"It's now 08:52 PM"

In [85]:
now.strftime('Today is %D')

'Today is 12/28/18'

> 如果类没有定义 `__format__` 方法，从 `object` 继承的方法会返回 `str(my_object)`，但是如果传入格式说明符就会抛出 `TypeError`

In [86]:
format(v1)

'(3.0, 4.0)'

In [87]:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

***关于自定义格式化的代码和测试，和之后的内容一起给出（不然代码会重复冗长）***

----

## 5、可散列的Vector